In [1]:
import re
from nltk.corpus import stopwords
from gensim.models import word2vec
import pickle

In [2]:
def trecPreprocessing(fname):
    with open(fname, 'r') as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    content = [w.replace('\t', ' ') for w in content]

    trec = []

    tmp_trec = []
    tmp_dic = {"p": [], "n": []}

    for index in range(len(content)):
        if ("<QApairs id=" in content[index] and index != 1):
            # If questions have no positive or no negative ans, discard it.
            if ((len(tmp_dic["p"]) != 0) and (len(tmp_dic["n"]) != 0)):
                tmp_trec.append(tmp_dic)
                trec.append(tmp_trec)

            tmp_trec = []
            tmp_dic = {"p": [], "n": []}

        elif (content[index] == "<question>"):
            tmp_trec.append(content[index + 1].lower())
        elif (content[index] == "<positive>"):
            tmp_dic["p"].append(content[index + 1].lower())
        elif (content[index] == "<negative>"):
            tmp_dic["n"].append(content[index + 1].lower())
            
    # Last round
    if ((len(tmp_dic["p"]) != 0) and (len(tmp_dic["n"]) != 0)):
        tmp_trec.append(tmp_dic)
        trec.append(tmp_trec)
    
    f = open("trec_train.pkl", "wb")
    pickle.dump(trec, f)
    f.close()
    
    return trec

In [3]:
def tokenize(text):
    pattern = re.compile('[a-zA-Z]+')
    return (match.group(0) for match in pattern.finditer(text))

#print(list(tokenize("As a member of the white master tribe of South Africa , the author grew up thinking himself a `` Just White Man . ")))

def genWord2vecModel(trec_train):
    # Stopwords from NLTK
    sw = stopwords.words("english")
    corpus = []
    for e in trec_train:
        tmp_corpus = []
        
        q = list(tokenize(e[0]))
        p = e[1]['p']
        n = e[1]['n']
        
        for token in q:
            if (token not in sw):
                tmp_corpus.append(token)
                
        for single_p in p:
            token_list = list(tokenize(single_p))
            for token in token_list:
                if (token not in sw):
                    tmp_corpus.append(token)  
                    
        for single_n in n:
            token_list = list(tokenize(single_n))
            for token in token_list:
                if (token not in sw):
                    tmp_corpus.append(token)  
                
        corpus.append(tmp_corpus)
    
    vec_model = word2vec.Word2Vec(corpus, sg=1, iter=4, size=250)#丟進word2vec
    vec_model.save("word2vec.model")
    
    return vec_model

In [4]:
def wordIndexing(vec_model):
    word_index = {} # 把WORD2VEC DIC 裡面的詞編號 

    count = 1
    for key in vec_model.wv.vocab.keys():
        word_index[key] = count
        count += 1
    
    #print("word_index = ", word_index)
    
    f = open("word_index_dic.pkl", "wb")
    pickle.dump(word_index, f)
    f.close()

In [5]:
trec_train = trecPreprocessing("train2393.cleanup.xml")

In [6]:
vec_model = genWord2vecModel(trec_train)

In [7]:
wordIndexing(vec_model)